# Лабораторная работа №4
Шевченко Дарья, Толстых Мария P3230

Вариант 1

1. построить модель линейной регрессии с указанными параметрами своими средствами(вместе со свободным коэффициентом), то есть пользоваться готовыми линейными моделями нельзя (максимум – для сравнения своего результата с готовой реализации, плюс можно использовать NumPy для матричных вычислений или специальные функции, решающие оптимизационные задачи);
2. рассчитать оценки наименьших квадратов для параметров и остаточной дисперсии;
3. вычислить коэффициент детерминации;
4. построить доверительные интервалы для параметров модели и остаточной дисперсии;

Зависимая переменная – популярность, предикторы – продолжительность, ”танцевальность”, ”энергичность”.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import t
from scipy.stats import f
from scipy.stats import chi2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Линейная регрессия выражается в следующей форме:

$Y = $$β_0$ + $β_1$$X_1$ + ... + $β_n$$X_n$ + $ɛ$

где  $X= $($X_1$, ..., $X_n$) - набор независимых переменных

n - число предсказаний

$β_0$, $β_1$, ..., $β_n$ - коэффициенты регрессии

$ɛ$ - случайная ошибка

In [ ]:
df = pd.read_csv('sample_data/song_data.csv')
Y = np.array(df['song_popularity'])
duration = np.array(df['song_duration_ms'])
danceability = np.array(df['danceability'])
energy = np.array(df['energy'])
X = np.array([duration, danceability, energy, np.ones(len(duration))]).T

Коэффициенты регрессии
c оцениваются методом наименьших квадратов.
Рассчитаем оценки наименьших квадратов для параметров.

1.

  $y = Xс+ϵ$

2. Минимизация суммы квадратов ошибок:

  $S(с) = ∑{(Y_i-X_iс)^2} = (Y-Xс)^T(Y-Xс) = Y^TY-2с^TX^TY+с^TX^TXс$

3. Производная:

  $\frac{δS(с)}{δс}=\frac{δ}{δс}(Y^TY-2с^TX^TY+с^TX^TXс)$

  $\frac{δS(с)}{δс}=-2X^TY+2X^TXс$

4. Минимум

  $-2X^TY+2X^TXс = 0$

5. Оценка
c вычисляется по формуле:
$\widehat{c}$ = $(X^TX)^{-1}$$X^T$$Y$

In [ ]:
C = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)
print("Коэффициенты регрессии:", C[0], C[1], C[2], C[3])

Коэффициенты регрессии: -2.8502021545008635e-06 14.478199189010326 -0.2567102836253759 44.609661234376055


In [ ]:
regression = LinearRegression().fit(X, Y)
print("Коэффициенты регрессии:",regression.coef_[0], regression.coef_[1], regression.coef_[2], regression.intercept_)

Коэффициенты регрессии: -2.850202154524885e-06 14.478199189009976 -0.25671028362546977 44.60966123438163


Оценка остаточной дисперсии

${S}^2 = \cfrac{\sum_{i=1}^n(y_{i}-\hat y_{i})^2}{n - p - 1}$


In [ ]:
n = len(Y)
p = X.shape[1] - 1
residuals = X @ C - Y
Ssq = np.sum(residuals**2)
var = Ssq / (n - p - 1)
print("Остаточная дисперсия:", var)

Остаточная дисперсия: 474.68050520294605


Коэффициент детерминации:
$$R^2 =\frac{\sigma^2}{\sigma^2_y}= \cfrac{\sum\limits^n_{i = 1}(Y_i - \hat{Y})^2}{\sum\limits^n_{i = 1}(Y_i - \overline{Y})^2}$$   

Коэффициент показывает, какую долю общей вариации в данных объясняет модель

In [ ]:
numerator = 0
Yhat = X @ C
for i in range(n):
    numerator += (Y[i] - Yhat[i]) ** 2

denom = 0
for i in range(n):
    denom += (Y[i] - np.mean(Y)) ** 2


R = numerator / denom
print("Коэффициент детерминации:", 1 - R)
print(Yhat, Y)

Коэффициент детерминации: 0.010946570823361368
[50.86806954 51.61956842 54.50075128 ... 54.41671742 50.58726567
 53.22629392] [73 66 76 ... 23 55 60]


Найдем доверительный интервал для параметров модели

$\frac{\widehat {c}_i - c_i}{\sqrt{(X^TX)^{-1}S^2(\widehat {c})}} \sim T(n - p -1)$

Выразим отсюда коэффициенты:

$\widehat{c}_i - T_{(1-α/2)(n-p-1)} \sqrt{(X^TX)^{-1}S^2(\widehat{c})} < c_i < \widehat{c}_i + T_{(1 - α/2)(n-p-1)}\sqrt{(X^TX)^{-1}S^2(\widehat{c})}$

Формула для подсчета доверительного интервала остаточной дисперсии:

$\frac{S^2(\widehat{c})}{σ} \sim χ^2(n-p-1)$

Выразим остаточную дисперсию:

$\frac{(n-p-1)S^2(\widehat{c})}{χ^2_{1-α/2(n-p-1)}} < σ^2 < \frac{(n-p-1)S^2(\widehat{c})}{χ^2_{α/2(n-p-1)}}$

${χ^2_{α/2(n-p-1)}}$ критические значения χ^2 распределения для уровня значимости
α и
n−p−1 степеней свободы

In [ ]:
alpha = 0.05
left = []
right = []

def count_borders(i):
    left.append(C[i] + t.ppf(1 - alpha / 2, n - p - 1) * np.sqrt(np.linalg.inv(np.dot(X.T, X))[i][i]) * Ssq)
    right.append(C[i] - t.ppf(1 - alpha / 2, n - p -1) * np.sqrt(np.linalg.inv(np.dot(X.T, X))[i][i]) * Ssq)

for i in range(3):
    count_borders(i)

In [ ]:
print("Доверительные интервалы для параметров модели:")
print(left)
print(right)

print("Доверительные интервалы остаточной дисперсии:")
print((n-p-1) * var / chi2.ppf(1 - alpha / 2, n - p - 1))
print((n-p-1) * var / chi2.ppf(alpha / 2, n - p - 1))

print(C[0], C[1], C[2], C[3])

Доверительные интервалы для параметров модели:
[2.153991407200194, 820372.6600631613, 599768.9143855638]
[-2.1539971076045026, -820343.7036647832, -599769.427806131]
Доверительные интервалы остаточной дисперсии:
465.23655458059085
484.41633176247376
-2.8502021545008635e-06 14.478199189010326 -0.2567102836253759 44.609661234376055
